# Анализ поведения пользователей мобильного приложения

**Описание задачи**\
В данной работе представлены результаты проведения А/А/В-теста пользователей мобильного приложения для продажи продуктов питания за период с 25 июля по 7 августа 2019 года. Тестирование проводилось для оценки изменений, связанных с изменением шрифта в приложении. Пользователи были разделены на три группы: 246 и 247 — контрольные группы со старым шрифтом в приложении, а 248 — экспериментальная группа с новым шрифтом в приложении.

**План работы**
1. Предобработка данных
- переименование столбцов
- поиск явных/неявных дубликатов
- добавление новых столбцов

2. Изучение и проверка данных

3. Изучение воронки событий

4. Изучение результатов эксперимента
- Расчёт статистической значимости различий в конверсии пользователей между 2 контрольными группами со старыми шрифтами
- Расчёт статистической значимости различий в конверсии пользователей между каждой контрольной группой со старыми шрифтами и группой с новыми шрифтами
- Расчёт статистической значимости различий в конверсии пользователей между объеденёнными контрольными группыми со старыми шрифтами и группой с новыми шрифтами

5. Подготовка общего вывода

**Описание данных**
1. logs_exp.csv - логи действий пользователей, где:
* EventName — название события
* DeviceIDHash — уникальный идентификатор пользователя
* EventTimestamp — время события
* ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная

# Общий вывод
- В процессе предобработки данных были удалены дубликаты. Из датасета было удалено 0.2% данных. В датасете не было обнаружено пропусков. В таблице были добавлены столбцы с датой.
- В датасете присутствуют 5 событий - MainScreenAppear, OffersScreenAppear, CartScreenAppear, PaymentScreenSuccessful, Tutorial. Наиболее популярным является MainScreenAppear, наименее популярным Tutorial
- Было определено, что наибольшее количество пользователей теряется на шаге перехода с главной страницы на страницу с предложениями. От первого события до оплаты примерно половина пользователей

**Оценка результатов А/А/В-теста**
- При расчёте статистической значимости было определено, что нет оснований считать контрольные группы разными.
- Нет статистически значимого различия по конверсии пользователей между объеденённой контрольной группой со старым шрифтом и группой с изменённым шрифтом.

Результаты теста не показали улучшение метрик (конверсия в целевые действия). Изменение шрифтов никак не повлияло на поведение пользователей.

**Рекомендации**\
Для увеличения конверсии в просмотр страницы с предложениями можно выполнить следующие действия:
1. Сделать кнопку с переходом на экран с предложениями более заметной
2. Добавить всплывающее окно, показывающее текущие акции и предложения

## Изучение общей информации 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
import scipy.stats as stats
import plotly.express as px
import math as mth
import seaborn as sb

In [2]:
try:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/logs_exp.csv', sep='\t')

except:
    data = pd.read_csv('/my_folder_with_files/logs_exp.csv', sep='\t')

In [3]:
data.head()

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


Вывод: пропущенные значения в датасетах не обнаружены.

## Предобработка данных

### Переименование столбцов

In [5]:
data.columns = ['event_name', 'user_id', 'event_timestamp', 'group_id']

In [6]:
data.sample()

,event_name,user_id,event_timestamp,group_id
216245,MainScreenAppear,1067090061368734284,1565156600,246


### Устранение дубликатов

In [7]:
#поиск уникальных значений в столбцах
data['event_name'].unique()

array(['MainScreenAppear', 'PaymentScreenSuccessful', 'CartScreenAppear',
       'OffersScreenAppear', 'Tutorial'], dtype=object)

Вывод: неявные дубликаты не обнаружены

In [8]:
#поиск явных дубликатов
data.duplicated().sum()

413

Вывод: В датасете обнаружено 413 дубликатов. Для дальнейшей работы их необходимо удалить.

In [9]:
#удаление явных дубликатов
data = data.drop_duplicates().reset_index(drop=True)

In [10]:
#расчёт количества удалённых данных
old_data = pd.read_csv('https://code.s3.yandex.net/datasets/logs_exp.csv', sep='\t')
print(f"Из датасета было удалено {round((1 - round((data.shape[0] / old_data.shape[0]), 3)) * 100, 2)}% данных")

Из датасета было удалено 0.2% данных


### Добавление новых столбцов

In [11]:
data['date_time'] = pd.to_datetime(data['event_timestamp'], unit = 's')
data['date'] = data['date_time'].astype('datetime64[D]') 
data.sample()

TypeError: Cannot cast DatetimeArray to dtype datetime64[D]

## Изучение и проверка данных

In [ ]:
#расчёт количества событий в логе
print('Количество событий в логе -',(data['event_name'].count()))
data.groupby('event_name').agg({'user_id':'count'}).reset_index()

Перед расчётом среднего количества событий на пользователя, необходимо проверить наличие аномальных значений.

In [ ]:
#создание сводной таблицы с расчётом количества событий для каждого пользователя
data_grouped = data.groupby('user_id').agg({'event_name':'count'}).reset_index()

data_grouped['event_name'].describe()

Вывод: стандартное отклонение 65, что говорит о наличии крупных аномалий количества событий у некоторых пользователей. Соответственно, для расчётом количества событий на пользователя необходимо использовать медианное значение.

In [ ]:
#расчёт пользователей в данных
print('Количество уникальных пользователей в датасете -',(data['user_id'].nunique()))

#расчёт медианного количества событий на пользователя
print(f"В среднем на пользователя приходится {data_grouped['event_name'].median()} событий")

#определение минимальной и максимальной даты в датасете
print('Минимальная дата в логе -',(data['date_time'].min()))
print('Максимальная дата в логе -',(data['date_time'].max()))

In [ ]:
#построение гистограммы по столбцу date_time
data['date_time'].hist(bins=80, figsize=(15,5))
plt.title('Гистограмма событий в зависимости от даты и времени')
plt.xlabel('Дата', fontsize=13)
plt.ylabel('Количество событий', fontsize=13);

Вывод: Исходя из анализа гистограммы, можно сделать вывод о том, что полные данные присутствуют за период с 1 по 7 августа (включительно).

In [ ]:
#удаление событий, случившихся до 1 августа 2019
data_new = data.query('date_time >= "2019-08-01"')

#расчёт количества удалённых пользователей и событий
print('Количество удалённых событий равно',(data.shape[0]-data_new.shape[0]),'Это составляет', round((data.shape[0]-data_new.shape[0])*100/data.shape[0],2), '% от изначального количества событий') 
print('Количество удалённых уникальных пользователей равно',(data['user_id'].nunique()-data_new['user_id'].nunique()),'Это составляет', round((data['user_id'].nunique()-data_new['user_id'].nunique())*100/data['user_id'].nunique(),2), '% от изначального количества пользователей') 


#проверка наличия пользователей из 3х групп в новом датасете
data_new['group_id'].unique()

Вывод: В новом датасете присутствуют пользователи из всех трёх экспериментальных групп.

## Изучение воронки событий 

In [ ]:
#группировка данных по событию для расчёта частоты совершения этого события
events_frequency = data_new.groupby('event_name').agg({'user_id':'count'}).reset_index()\
                           .sort_values(by='user_id', ascending=False)

events_frequency.columns = ['event_name', 'quantity']
events_frequency

Вывод: В логах присутствуют такие события, как MainScreenAppear, OffersScreenAppear, CartScreenAppear, PaymentScreenSuccessful, Tutorial. \
Чаще всего присутствует событие MainScreenAppear, которое означает переход на главную страницу. \
Реже всего встречается событие Tutorial.

In [ ]:
#создание таблицы, отображающей количество пользователей для каждого события и долю пользователей, которые хоть раз совершали событие
events_table = data_new.groupby('event_name').agg({'user_id':'nunique'}).reset_index()\
                         .sort_values(by='user_id', ascending=False)
events_table.columns = ['event_name', 'users_quantity']
events_table['ratio'] = round((events_table['users_quantity']/data_new['user_id'].nunique()), 2)
events_table

Вывод:\
Скорее всего, события происходят в следующей последовательности:  
MainScreenAppear->OffersScreenAppear->CartScreenAppear->PaymentScreenSuccessful \
главная страница->страница с преложениями->корзина->страница с подтверждением оплаты \
Минимальная доля пользователей совершает событие Tutotial (всего 11%). Данное событие не будет использовано для дальнейшего анализа.


In [ ]:
#расчёт доли пользователей, переходящих на следующий шаг воронки
logs_funnel = events_table.query('event_name != "Tutorial"').reset_index(drop=True)
logs_funnel['conversion, %'] = round((logs_funnel['users_quantity']*100 / logs_funnel['users_quantity'].shift(1)), 1)
logs_funnel = logs_funnel.fillna(100) 
logs_funnel

In [ ]:
#визуализация воронки
funnel_data = dict(values=logs_funnel['conversion, %'],
            labels=logs_funnel['event_name'])
fig = px.funnel(funnel_data, y='labels', x='values', title='Воронка событий')
fig.show()


In [ ]:
print('Наибольшее количество пользователей теряется на шаге перехода с главной страницы на страницу с предложениями.')
print('От первого события до оплаты доходит',(round(logs_funnel['users_quantity'][3]*100/logs_funnel['users_quantity'][0])), '% пользователей')

## Изучение результатов эксперимента

In [ ]:
#расчёт количества пользователей в каждой экспериментальной группе
users_quantity_by_group = data_new.query('event_name !="Tutorial"').groupby('group_id').agg({'user_id':'nunique'})\
                                  .reset_index()
users_quantity_by_group

Вывод: в группе 246 находится 2483 пользователя, в группе 247 - 2512, в группе 248 - 2535 пользователя.

In [ ]:
#проверка наличия уникальных пользователей в 2х или 3х группах одновременно
data_new.query('event_name !="Tutorial"').groupby('user_id').agg({'group_id':'nunique'}).reset_index().query('group_id > 1').count()


Вывод: пользователей, которые находятся в нескольких группах одновременно, не обнаружено.

### Анализ А/А/B-теста

In [ ]:
#создание сводной таблицы с числом уникальных пользователей по событиям и контрольным группам
combined_data = data_new.query('event_name !="Tutorial"').groupby(['group_id','event_name'])\
                                                         .agg({'user_id':'nunique'}).reset_index()

combined_data

In [ ]:
#функция для проведения z-теста
def get_z_test(event, group1, group2, alpha):
    successes = np.array([combined_data.query('group_id==@group1 & event_name == @event')['user_id'], 
                      combined_data.query('group_id==@group2 & event_name == @event')['user_id']])
    trials = np.array([users_quantity_by_group.query('group_id == @group1')['user_id'], 
                   users_quantity_by_group.query('group_id == @group2')['user_id']])
    alpha = alpha
    
    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]
    
    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]
    
    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
    
    # разница пропорций в датасетах
    difference = p1 - p2
    
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference[0] / mth.sqrt(p_combined[0] * (1 - p_combined[0]) * (1/trials[0] + 1/trials[1]))
    
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = stats.norm(0, 1) 
    
    p_value = (1 - distr.cdf(abs(z_value))) * 2

    Shidaka_alpha = 1 - (1 - alpha)**(1/16)  # метод Шидака для корректировки значения требуемого уровня значимости, учитывая 16 сравнений
    
    print('p-значение: ', p_value)

    if p_value < Shidaka_alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print(
        'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными'
    )

### Проведение z-теста для всех событий групп 246 и 247

Прежде всего, необходимо сформулировать гипотезы:
1. Нулевая (Ho): доля количества уникальных пользователей на выбранном этапе относительно общего числа пользователей в первой группе равна доли количества уникальных пользователей на выбранном этапе относительно общего числа пользователей во второй группе 
2. Альтернативная (Н1): доля количества уникальных пользователей на выбранном этапе относительно общего числа пользователей в первой группе не равна доли количества уникальных пользователей на выбранном этапе относительно общего числа пользователей во второй группе 

alpha = 0.05 - уровень статистической значимости

In [ ]:
get_z_test('MainScreenAppear', 246, 247, 0.05)

In [ ]:
get_z_test('CartScreenAppear', 246, 247, 0.05)

In [ ]:
get_z_test('OffersScreenAppear', 246, 247, 0.05)

In [ ]:
get_z_test('PaymentScreenSuccessful', 246, 247, 0.05)

Вывод: 
1. Исходя из результатов можно сделать вывод, что нет оснований считать контрольные выборки групп 246 и 247 разными. Равные доли свидетельствуют о том, что контрольные группы выбраны верны.
2. Самым популярным событием является показ главного экрана сайта.

### Проведение z-теста для всех событий групп 246 и 248

Прежде всего, необходимо сформулировать гипотезы:
1. Нулевая (Ho): доля количества уникальных пользователей на выбранном этапе относительно общего числа пользователей в первой группе равна доли количества уникальных пользователей на выбранном этапе относительно общего числа пользователей во второй группе 
2. Альтернативная (Н1): доля количества уникальных пользователей на выбранном этапе относительно общего числа пользователей в первой группе не равна доли количества уникальных пользователей на выбранном этапе относительно общего числа пользователей во второй группе 

alpha = 0.05 - уровень статистической значимости

In [ ]:
get_z_test('MainScreenAppear', 246, 248, 0.05)

In [ ]:
get_z_test('CartScreenAppear', 246, 248, 0.05)

In [ ]:
get_z_test('OffersScreenAppear', 246, 248, 0.05)

In [ ]:
get_z_test('PaymentScreenSuccessful', 246, 248, 0.05)

Вывод:  Р-value для всех 4х событий больше 0.05, поэтому нулевую гипотезу о равенстве долей успехов между двумя группами нельзя отвергнуть. Статистически значимых отличий в долях между группами нет.

### Проведение z-теста для всех событий групп 247 и 248

Прежде всего, необходимо сформулировать гипотезы:
1. Нулевая (Ho): доля количества уникальных пользователей на выбранном этапе относительно общего числа пользователей в первой группе равна доли количества уникальных пользователей на выбранном этапе относительно общего числа пользователей во второй группе 
2. Альтернативная (Н1): доля количества уникальных пользователей на выбранном этапе относительно общего числа пользователей в первой группе не равна доли количества уникальных пользователей на выбранном этапе относительно общего числа пользователей во второй группе 

alpha = 0.05 - уровень статистической значимости

In [ ]:
get_z_test('MainScreenAppear', 247, 248, 0.05)

In [ ]:
get_z_test('CartScreenAppear', 247, 248, 0.05)

In [ ]:
get_z_test('OffersScreenAppear', 247, 248, 0.05)

In [ ]:
get_z_test('PaymentScreenSuccessful', 247, 248, 0.05)

Вывод: Р-value для всех 4х событий больше 0.05, поэтому нулевую гипотезу о равенстве долей успехов между двумя группами нельзя отвергнуть. Статистически значимых отличий в долях между группами нет.

### Проведение z-теста для всех событий групп 248 и 249 (объединённая группа из 246, 247)

Прежде всего, необходимо сформулировать гипотезы:
1. Нулевая (Ho): доля количества уникальных пользователей на выбранном этапе относительно общего числа пользователей в первой группе равна доли количества уникальных пользователей на выбранном этапе относительно общего числа пользователей во второй группе 
2. Альтернативная (Н1): доля количества уникальных пользователей на выбранном этапе относительно общего числа пользователей в первой группе не равна доли количества уникальных пользователей на выбранном этапе относительно общего числа пользователей во второй группе 

alpha = 0.05 - уровень статистической значимости

In [ ]:
#подготовка таблицы для проведения теста
combined_data.loc[13] = (249, 'CartScreenAppear', combined_data.loc[0]['user_id']+combined_data.loc[4]['user_id'])
combined_data.loc[14] = (249, 'MainScreenAppear', combined_data.loc[1]['user_id']+combined_data.loc[5]['user_id'])
combined_data.loc[15] = (249, 'OffersScreenAppear', combined_data.loc[2]['user_id']+combined_data.loc[6]['user_id'])
combined_data.loc[16] = (249, 'PaymentScreenSuccessful', combined_data.loc[3]['user_id']+combined_data.loc[7]['user_id'])
combined_data


In [ ]:
#подготовка таблицы для проведения теста
users_quantity_by_group.loc[3] = (249, users_quantity_by_group.loc[0]['user_id']+users_quantity_by_group.loc[1]['user_id'])


In [ ]:
get_z_test('MainScreenAppear', 248, 249, 0.05)

In [ ]:
get_z_test('CartScreenAppear', 248, 249, 0.05)

In [ ]:
get_z_test('OffersScreenAppear', 248, 249, 0.05)

In [ ]:
get_z_test('PaymentScreenSuccessful', 248, 249, 0.05)

Вывод:  Р-value для всех 4х событий больше 0.05, поэтому нулевую гипотезу о равенстве долей успехов между двумя группами нельзя отвергнуть. Статистически значимых отличий в долях между группами нет.

# Общий вывод
- В процессе предобработки данных были удалены дубликаты. Из датасета было удалено 0.2% данных. В датасете не было обнаружено пропусков. В таблице были добавлены столбцы с датой.
- В датасете присутствуют 5 событий - MainScreenAppear, OffersScreenAppear, CartScreenAppear, PaymentScreenSuccessful, Tutorial. Наиболее популярным является MainScreenAppear, наименее популярным Tutorial
- Было определено, что наибольшее количество пользователей теряется на шаге перехода с главной страницы на страницу с предложениями. От первого события до оплаты примерно половина пользователей

**Оценка результатов А/А/В-теста**
- При расчёте статистической значимости было определено, что нет оснований считать контрольные группы разными.
- Нет статистически значимого различия по конверсии пользователей между объеденённой контрольной группой со старым шрифтом и группой с изменённым шрифтом.

Результаты теста не показали улучшение метрик (конверсия в целевые действия). Изменение шрифтов никак не повлияло на поведение пользователей.

**Рекомендации**\
Для увеличения конверсии в просмотр страницы с предложениями можно выполнить следующие действия:
1. Сделать кнопку с переходом на экран с предложениями более заметной
2. Добавить всплывающее окно, показывающее текущие акции и предложения